# How to optimize a quantum kernel

One of the main challenges regarding quantum kernels is the need to choose the appropriate ansatz for each different problem. When little-to-no information about a certain task is known, we can use techniques to attempt to construct a suitable quantum kernel following an optimization problem. There are two main possible approaches:

1. Choosing an ansatz where some of the parameters are the usual features, while others are freely tunable parameters that are optimized according to some cost function via a stochastic gradient descent-based algorithm.
2. Avoid making any choice and let an optimization algorithm pick the entire quantum circuit.

In this tutorial, we will demonstrate how to easily implement the first technique. Furthermore, we will also discuss how to leverage _quask_ built-in features to implement the second technique. Finally, we will show how it is possible to efficiently achieve the best-performing linear combination of these kernels when dealing with multiple different kernels.

## Optimization of quantum kernels in _quask_

The package quask.optimizer allows to define optimization procedure over quantum kernels. The main interface is the class BaseKernelOptimizer, which needs:
* the kernel function;
* a kernel evaluator to use as a cost function;


## Gradient descent-based optimization of a quantum kernel


## Combinatorial optimization of a quantum kernel

### Bayesian optimizer

### Greedy optimizer

### Meta-heuristic optimizer

### Reinforcement learning optimizer

### Add your own criteria

## Multiple kernel learning

## References